In [75]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [76]:
# create fully connected (fc) network
class Network(nn.Module): # (28 x 28 input imgs => 784 dim input)
    def __init__(self, num_classes):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=784, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

# create convolutions network
class ConvNetwork(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(ConvNetwork, self).__init__()
        # 1 input channel for grayscale 
        input_dim, pooling_coeff, pooling_count = 28, 2, 2 
        conv1_out_dim, conv1_kernel = 8, 5
        conv2_out_dim, conv2_kernel = pooling_coeff * conv1_out_dim, conv1_kernel // pooling_coeff
        
        # convolve input data
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=conv1_out_dim, kernel_size=(conv1_kernel,conv1_kernel), stride=(1,1), padding=(1,1))
        self.pool = nn.AvgPool2d(kernel_size=(pooling_coeff,pooling_coeff), stride=(pooling_coeff,pooling_coeff))
        self.conv2 = nn.Conv2d(in_channels=conv1_out_dim, out_channels=conv2_out_dim, kernel_size=(conv2_kernel,conv2_kernel), stride=(1,1), padding=(1,1))
        # fully-connected layers
        fc1_input = int(conv2_out_dim * pow(input_dim * pow(2.0, -pooling_count), pooling_count))
        self.fc1 = nn.Linear(fc1_input, fc1_input // 2)
        self.fc2 = nn.Linear(fc1_input // 2, fc1_input // 2)
        self.out = nn.Linear(fc1_input // 2, num_classes)

    def forward(self, x):
        # convolve:
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        # fully connected layers:
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [77]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# set hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 20
learning_rate = 0.001
batch_size = 64


In [78]:
# load training data
training_data = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
training_loader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
# load testing data
testing_data = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
testing_loader = DataLoader(dataset=testing_data, batch_size=batch_size, shuffle=True)

In [79]:
# initialize network model
model = ConvNetwork(in_channels=in_channels, num_classes=num_classes).to(device)
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [81]:
# train network
for epoch in range(0, num_epochs):
    for batch_idx, (data, targets) in enumerate(training_loader): 
        # get data for batch
        data = data.to(device)
        targets = targets.to(device)

        # forward propagate
        loss = criterion(model(data), targets)
        # back-propagate
        optimizer.zero_grad()
        loss.backward()
        # gradient-descent
        optimizer.step()
        

In [ ]:
# check accuracy
num_correct, num_samples = 0, 0
model.eval() # turn model to evaluation mode from training

with torch.no_grad():
    for (x, y) in testing_loader:
        x = x.to(device)
        y = y.to(device)
        
        result = model(x) #shape: 64 X 10
        _,predictions = result.max(1)
        num_correct += (predictions == y).sum()
        num_samples += predictions.size(0)
    
    # print results
    print(f"Found {num_correct} / {num_samples} with accuracy of {format(float(num_correct/num_samples), '.3f')}")
model.train()

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [64, 784]